In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import os

np.random.seed(42)

## Simulate Baseline vs Optimized Performance

In [ ]:
# Generate baseline (without AI) simulation
duration = 3600  # 1 hour
timesteps = np.arange(0, duration, 10)  # 10-second intervals

# Baseline: random unoptimized operations
baseline_power = np.random.normal(loc=45, scale=8, size=len(timesteps))
baseline_power = np.maximum(baseline_power, 20)  # Minimum 20 kW

# Optimized: smoother with lower average
optimized_power = np.random.normal(loc=35, scale=5, size=len(timesteps))
optimized_power = np.maximum(optimized_power, 18)  # Minimum 18 kW

# Calculate energy
baseline_energy = np.cumsum(baseline_power) * 10 / 3600  # kWh
optimized_energy = np.cumsum(optimized_power) * 10 / 3600  # kWh

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'timestamp': timesteps,
    'baseline_power': baseline_power,
    'optimized_power': optimized_power,
    'baseline_energy': baseline_energy,
    'optimized_energy': optimized_energy
})

print("Comparison Data:")
print(comparison_df.head(10))
print(f"\nBaseline - Total Energy: {baseline_energy[-1]:.2f} kWh")
print(f"Optimized - Total Energy: {optimized_energy[-1]:.2f} kWh")
print(f"Savings: {(1 - optimized_energy[-1]/baseline_energy[-1])*100:.1f}%")

## Performance Metrics Calculation

In [ ]:
def calculate_performance_metrics(baseline_data, optimized_data):
    """Calculate comprehensive performance metrics."""
    
    metrics = {
        'Mean Power (kW)': {
            'Baseline': np.mean(baseline_data),
            'Optimized': np.mean(optimized_data)
        },
        'Std Dev Power': {
            'Baseline': np.std(baseline_data),
            'Optimized': np.std(optimized_data)
        },
        'Peak Power (kW)': {
            'Baseline': np.max(baseline_data),
            'Optimized': np.max(optimized_data)
        },
        'Min Power (kW)': {
            'Baseline': np.min(baseline_data),
            'Optimized': np.min(optimized_data)
        },
        'Power Range (kW)': {
            'Baseline': np.max(baseline_data) - np.min(baseline_data),
            'Optimized': np.max(optimized_data) - np.min(optimized_data)
        }
    }
    
    return metrics

metrics = calculate_performance_metrics(baseline_power, optimized_power)

# Display as table
metrics_df = pd.DataFrame(metrics)
print("\nPerformance Metrics:")
print(metrics_df)

# Calculate percentage improvements
print("\nPercentage Improvements:")
mean_improvement = (metrics['Mean Power (kW)']['Baseline'] - metrics['Mean Power (kW)']['Optimized']) / metrics['Mean Power (kW)']['Baseline'] * 100
peak_reduction = (metrics['Peak Power (kW)']['Baseline'] - metrics['Peak Power (kW)']['Optimized']) / metrics['Peak Power (kW)']['Baseline'] * 100
stability_improvement = (metrics['Std Dev Power']['Baseline'] - metrics['Std Dev Power']['Optimized']) / metrics['Std Dev Power']['Baseline'] * 100

print(f"  Mean Power Reduction: {mean_improvement:.1f}%")
print(f"  Peak Power Reduction: {peak_reduction:.1f}%")
print(f"  Stability Improvement: {stability_improvement:.1f}%")

## Statistical Analysis

In [ ]:
# Perform paired t-test
t_stat, p_value = stats.ttest_rel(baseline_power, optimized_power)

print("Statistical Test Results (Paired t-test):")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_value:.6f}")
print(f"  Significant difference (p<0.05): {p_value < 0.05}")

# Effect size (Cohen's d)
mean_diff = np.mean(baseline_power) - np.mean(optimized_power)
pooled_std = np.sqrt((np.std(baseline_power)**2 + np.std(optimized_power)**2) / 2)
cohens_d = mean_diff / pooled_std

print(f"\nEffect Size (Cohen's d): {cohens_d:.4f}")
if abs(cohens_d) < 0.2:
    effect = "negligible"
elif abs(cohens_d) < 0.5:
    effect = "small"
elif abs(cohens_d) < 0.8:
    effect = "medium"
else:
    effect = "large"
print(f"Effect magnitude: {effect}")

## Visualizations

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Power over time
axes[0, 0].plot(timesteps, baseline_power, 'r-', alpha=0.7, label='Baseline', linewidth=2)
axes[0, 0].plot(timesteps, optimized_power, 'g-', alpha=0.7, label='Optimized', linewidth=2)
axes[0, 0].fill_between(timesteps, baseline_power, optimized_power, alpha=0.2, color='blue')
axes[0, 0].set_title('Power Consumption Over Time')
axes[0, 0].set_xlabel('Time (seconds)')
axes[0, 0].set_ylabel('Power (kW)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Cumulative energy
axes[0, 1].plot(timesteps, baseline_energy, 'r-', alpha=0.7, label='Baseline', linewidth=2)
axes[0, 1].plot(timesteps, optimized_energy, 'g-', alpha=0.7, label='Optimized', linewidth=2)
axes[0, 1].fill_between(timesteps, baseline_energy, optimized_energy, alpha=0.2, color='blue')
axes[0, 1].set_title('Cumulative Energy Consumption')
axes[0, 1].set_xlabel('Time (seconds)')
axes[0, 1].set_ylabel('Energy (kWh)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Distribution comparison
axes[1, 0].hist(baseline_power, bins=30, alpha=0.6, label='Baseline', color='red')
axes[1, 0].hist(optimized_power, bins=30, alpha=0.6, label='Optimized', color='green')
axes[1, 0].set_title('Power Distribution')
axes[1, 0].set_xlabel('Power (kW)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Box plot comparison
data_for_box = [baseline_power, optimized_power]
axes[1, 1].boxplot(data_for_box, labels=['Baseline', 'Optimized'])
axes[1, 1].set_title('Power Statistics Comparison')
axes[1, 1].set_ylabel('Power (kW)')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Plotly Interactive Comparison

In [ ]:
# Interactive Plotly visualization
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=timesteps, y=baseline_power,
    mode='lines',
    name='Baseline',
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=timesteps, y=optimized_power,
    mode='lines',
    name='Optimized',
    line=dict(color='green', width=2)
))

fig.update_layout(
    title='Baseline vs Optimized Power Consumption',
    xaxis_title='Time (seconds)',
    yaxis_title='Power (kW)',
    hovermode='x unified',
    height=600
)

fig.show()

## Export Results